In [46]:
from geopy.geocoders import Nominatim #geolocate query
from geopy.distance import geodesic #measure distance
from gmplot import gmplot
import requests, json
from itertools import permutations

In [47]:
def mark_location(destination):
    # to store the list of latitude and longitude for each city
    for i in range (0,len(destination)):
        dest = geolocator.geocode(destination[i])  # .geocode
        cityList.append((dest.latitude, dest.longitude))
        cityDict[destination[i]] = cityList[i]

In [48]:
def plot_location(kl):
    #mark locations (GMPlot)
    gmap = gmplot.GoogleMapPlotter(kl.latitude, kl.longitude,3)
    gmap.apikey = ""
    city_lats, city_lons = zip(*cityList) #* to unpack cityList
    #scatter plots
    gmap.scatter(city_lats, city_lons, '#3B0B39', size=40, marker=False)
    gmap.draw("map_for_ben.html")

In [49]:
# Measuring distances
#Method 1: Python Geocoding Toolbox

def measure_distance(start,destination):
    for i in destination:
        dest = geolocator.geocode(i) #.geocode
        print(" ")
        print("City Name: ",dest.address)
        print("Coordinates of city: ",dest.latitude, dest.longitude)

        # Measuring distance
        current_location = (start.latitude, start.longitude)
        destination_location = (dest.latitude, dest.longitude)
        print("Total distance:")
        print(geodesic(current_location, destination_location).kilometers)


In [50]:
#Method 2: Google Distance Matrix API
def land_transport(source,dest_name):
#    source = destination[0]
#    dest_name="Bangkok, Thailand"
    url = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=metric&'
    r = requests.get(url + 'origins= ' + source +
                         '&destinations= ' + dest_name +
                        '&avoid=tolls&departure_time=now&mode=driving&key=YourAPiKey')
    # mode - drving(default), walking, cycling ...
    # units - metrics (default)
    # avoid - tolls, highway (biases the result to more favorable routes)
    x = r.json()
    y = x.get('rows')
    z = y[0].get('elements')
    ans = z[0].get('duration')
    print("---------------------------------------------------")
    print("Duration if drive to Thailand: ")
    print("Extra option : avoid tolls")
    print(ans.get('text')+'\n')

In [51]:
#Plan shortest route
#permutation
#sum all shortest route
def permutation(lists):
    dest_list = list(permutations(lists))
    return dest_list

#city dict
def shortest_route(dest_list):
    index = 0
    min = 9999999999
    total_dist = [0] * len(dest_list)
    for k in range(0,len(dest_list)) :
        dest_list[k] = list(dest_list[k])   #convert frm tuple to list
        dest_list[k].insert(0,destination[0])
        for i in range (0, len(dest_list[k])-1):
            total_dist[k] += geodesic(cityDict.get(dest_list[k][i]), cityDict.get(dest_list[k][i+1])).kilometers
        if(total_dist[k]<min):
            min = total_dist[k]
            index = k
    print("Shortest distance is :")
    print(total_dist[index])
    return index

In [52]:
#driver code
if __name__ == "__main__":
    # 1. Mark Locations
    # Create a list - consisting of tuples
    cityList = []
    cityDict = {}
    destination = ['KLIA, Kuala Lumpur,Malaysia', 'Jakarta,Indonesia', 'Bangkok,Thailand', 'Taipei,Taiwan', 'Hong Kong',
                   'Tokyo,Japan', 'Beijing,China', 'Seoul,Korea']
    
    geolocator = Nominatim(user_agent="my app")
    mark_location(destination)
    kl = geolocator.geocode(destination[0])
    plot_location(kl)

    # 2. Measuring distances
    #Method 1: Python Geocoding Toolbox
    print("\nDestination from KLIA to:")
    measure_distance(kl, destination[1:])

    #Method 2: Google distance matrix API 
    print("API Key needed*")
    print("\nAlternative to travel by land:")
    land_transport('KLIA, Kuala Lumpur,Malaysia','Bangkok,Thailand')
    print("")

    # 3. Find shortest path
    dest_list = permutation(destination[1:])
    index = shortest_route(dest_list)
    print("\nThe shortest route is: ")
    print(dest_list[index])



Destination from KLIA to:
 
City Name:  Daerah Khusus Ibukota Jakarta, Indonesia
Coordinates of city:  -6.1753942 106.827183
Total distance:
1177.3582420392047
 
City Name:  กรุงเทพมหานคร, เขตพระนคร, กรุงเทพมหานคร, 10200, ประเทศไทย
Coordinates of city:  13.7542529 100.493087
Total distance:
1181.968349658021
 
City Name:  臺北市, 信義區, 臺北市, 11008, Taiwan
Coordinates of city:  25.0375198 121.5636796
Total distance:
3226.9158493839436
 
City Name:  香港島 Hong Kong Island, 香港 Hong Kong, China 中国
Coordinates of city:  22.2793278 114.1628131
Total distance:
2510.632968396837
 
City Name:  東京都, 日本 (Japan)
Coordinates of city:  35.6828387 139.7594549
Total distance:
5320.78098753437
 
City Name:  北京市, 东城区, 北京市, 100010, China 中国
Coordinates of city:  39.906217 116.3912757
Total distance:
4334.413369436286
 
City Name:  서울, 대한민국
Coordinates of city:  37.5666791 126.9782914
Total distance:
4604.070956482475
API Key needed*

Alternative to travel by land:
----------------------------------------------